In [1]:
import os
import ast
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

ROOT_PATH = os.getcwd()
DATA_LOAD_FILE = os.path.join(ROOT_PATH, "data/StackOverFlowEmbedded.csv.gz")

df = pd.read_csv(DATA_LOAD_FILE)
#df["BertEmbeddingsMean"] = df["BertEmbeddingsMean"].apply(ast.literal_eval)
#df["BertEmbeddingsSum"] = df["BertEmbeddingsSum"].apply(ast.literal_eval)

#df["FastTextEmbeddings"] = df["FastTextEmbeddings"].apply(ast.literal_eval)
#df["UseEmbeddings"] = df["UseEmbeddings"].apply(ast.literal_eval)
df

,Date,Title,Tags,Score,Label,TitleClean,TitleCleanTokenised,BertEmbeddingsMean,BertEmbeddingsSum
0,2016-02-04 12:49:19,Java contains vs anyMatch behaviour,"java, java-stream, equality",56,java,java contain v anymatch behaviour,"['java', 'contain', 'v', 'anymatch', 'behaviour']","[-0.23472099006175995, -0.24564139544963837, 0...","[-9.154118537902832, -9.5800142288208, 6.10894..."
1,2011-05-04 19:52:46,Getting random numbers in Java,"java, random",557,java,get random number in java,"['get', 'random', 'number', 'in', 'java']","[-0.2763572931289673, -0.42447564005851746, 0....","[-10.777934074401855, -16.554550170898438, 0.3..."
2,2012-03-25 17:38:01,Hibernate generates negative id values when us...,"java, hibernate, jpa, jboss7.x, jpa-2.0",53,java,hibern gener neg id valu when use a sequenc,"['hibern', 'gener', 'neg', 'id', 'valu', 'when...","[-0.349016398191452, -0.19550490379333496, -0....","[-13.611639976501465, -7.624691009521484, -0.3..."
3,2010-08-02 17:35:16,How to get the separate digits of an int number?,"java, integer, modulo",196,java,how to get the separ digit of an int number,"['how', 'to', 'get', 'the', 'separ', 'digit', ...","[-0.25365835428237915, -0.3813159167766571, -0...","[-9.892675399780273, -14.871320724487305, -7.0..."
4,2011-06-10 07:52:16,Eclipse plugin for generating a class diagram,"java, eclipse, plugins, uml, class-diagram",112,java,eclips plugin for gener a class diagram,"['eclips', 'plugin', 'for', 'gener', 'a', 'cla...","[-0.251219242811203, -0.4725949764251709, -0.0...","[-9.797550201416016, -18.431203842163086, -1.6..."
...,...,...,...,...,...,...,...,...,...
9995,2014-06-29 03:25:18,How to call Type Methods within an instance me...,"ios, class, methods, types, swift",61,ios,how to call type method within an instanc method,"['how', 'to', 'call', 'type', 'method', 'withi...","[-0.17384220659732819, -0.3111932873725891, -0...","[-6.953688144683838, -12.447731018066406, -8.0..."
9996,2014-07-31 14:55:53,What is NSLayoutConstraint &quot;UIView-Encaps...,"ios, uitableview, cocoa-touch, autolayout, ios...",306,ios,what is nslayoutconstraint quotuiviewencapsula...,"['what', 'is', 'nslayoutconstraint', 'quotuivi...","[-0.1488007754087448, 0.2860782742500305, 0.34...","[-5.952031135559082, 11.443131446838379, 13.78..."
9997,2015-03-17 20:37:23,How do I fix the xcrun unable to find simctl e...,"ios, xcode, xcrun",359,ios,how do i fix the xcrun unabl to find simctl error,"['how', 'do', 'i', 'fix', 'the', 'xcrun', 'una...","[-0.2691449224948883, -0.272336483001709, 0.01...","[-10.765796661376953, -10.89345932006836, 0.45..."
9998,2013-05-03 17:23:43,NSAttributedString background color and rounde...,"ios, objective-c, uiview, quartz-graphics, nsa...",76,ios,nsattributedstr background color and round corner,"['nsattributedstr', 'background', 'color', 'an...","[-0.3197779357433319, -0.11719034612178802, 0....","[-12.791117668151855, -4.6876139640808105, 7.8..."


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from optimum.onnxruntime import ORTModelForSequenceClassification

import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model_name = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True,)
model = AutoModelForSequenceClassification.from_pretrained(model_name,)

/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
df["LabelEncoded"] = LabelEncoder().fit_transform(df["Label"].values)
TrainDF, TestDF = train_test_split(df,train_size=0.7, random_state=42, stratify=df["LabelEncoded"].values)

TrainDF.reset_index(inplace=True)
TestDF.reset_index(inplace=True)

candidate_labels = TestDF["Label"].unique().tolist()



In [32]:
candidate_labels

['c#',
 'c++',
 'git',
 'html',
 'android',
 'ios',
 'python',
 'javascript',
 'java',
 'php']

In [12]:
from transformers import pipeline
import pandas as pd
import numpy as np

# Example DataFrame creation (replace this with your actual TestDF)
# TestDF = pd.DataFrame({'Title': ["Title 1", "Title 2"], 'Label': ["Label 1", "Label 2"]})

batch_size = 128

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Initialize the classifier pipeline
classifier = pipeline(
    task="zero-shot-classification", 
    model=model,
    tokenizer= tokenizer ,
    use_fast = True,
    batch_size = batch_size,
    framework = "pt",
)

# Your unique candidate labels
candidate_labels = TestDF["Label"].unique().tolist()

# Get your sequences
sequences = TestDF["Title"].values.tolist()

# Prepare batches
batches = [sequences[i:i + batch_size] for i in range(0, len(sequences), batch_size)]
BatchNumbers = len(batches)

# Classify each batch and collect the results
results = []
#y_hat =[]

for idx, batch in enumerate(batches):
    # Classify the entire batch at once
    batch_output = classifier(batch, candidate_labels, multi_label = False, batch_size=batch_size)
    results.append(batch_output)  # Collect results
    #y_hat.append(batch_output["labels"][0])
    print(f"Batch {idx+1} of {BatchNumbers}")

/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Batch 1 of 24
Batch 2 of 24
Batch 3 of 24
Batch 4 of 24
Batch 5 of 24
Batch 6 of 24
Batch 7 of 24
Batch 8 of 24
Batch 9 of 24
Batch 10 of 24
Batch 11 of 24
Batch 12 of 24
Batch 13 of 24
Batch 14 of 24
Batch 15 of 24
Batch 16 of 24
Batch 17 of 24
Batch 18 of 24
Batch 19 of 24
Batch 20 of 24
Batch 21 of 24
Batch 22 of 24
Batch 23 of 24
Batch 24 of 24


In [15]:
res_df = pd.DataFrame([i for res in results for i in res])
res_df

,sequence,labels,scores
0,Cross-platform file name handling in .NET Core,"[c#, c++, git, php, java, html, ios, android, ...","[0.28499728441238403, 0.1304446905851364, 0.10..."
1,Why use array size 1 instead of pointer?,"[c++, php, html, java, c#, python, ios, git, j...","[0.18912053108215332, 0.14287739992141724, 0.1..."
2,&quot;Resource not accessible by integration&q...,"[git, html, python, c#, c++, java, javascript,...","[0.46193403005599976, 0.13478082418441772, 0.1..."
3,How to use text-overflow ellipsis in an html i...,"[html, php, c#, java, javascript, c++, ios, py...","[0.9558219313621521, 0.009568645618855953, 0.0..."
4,java.lang.IllegalStateException: The specified...,"[java, c#, html, c++, android, git, php, ios, ...","[0.7532255053520203, 0.04914847016334534, 0.04..."
...,...,...,...
2995,Why does the new Java 8 Date Time API not have...,"[java, html, c++, c#, php, git, android, pytho...","[0.8284229040145874, 0.04380547255277634, 0.04..."
2996,Entitlements file do not match those specified...,"[c#, html, c++, python, php, android, ios, jav...","[0.17025446891784668, 0.14452692866325378, 0.1..."
2997,How to check if a Java 8 Stream is empty?,"[java, html, android, c#, c++, php, javascript...","[0.8730052709579468, 0.025802765041589737, 0.0..."
2998,Gitignore not working,"[git, c++, html, javascript, c#, java, python,...","[0.5811044573783875, 0.10151496529579163, 0.09..."


In [17]:
y_hat = res_df["labels"].apply(lambda x: x[0]).to_list()
y_hat

['c#',
 'c++',
 'git',
 'html',
 'java',
 'html',
 'ios',
 'java',
 'c++',
 'c#',
 'ios',
 'java',
 'git',
 'git',
 'java',
 'html',
 'git',
 'html',
 'java',
 'python',
 'html',
 'html',
 'html',
 'java',
 'c#',
 'php',
 'java',
 'java',
 'c++',
 'java',
 'java',
 'html',
 'php',
 'php',
 'php',
 'html',
 'c++',
 'html',
 'ios',
 'html',
 'html',
 'java',
 'git',
 'java',
 'java',
 'html',
 'java',
 'javascript',
 'html',
 'php',
 'html',
 'git',
 'android',
 'python',
 'c#',
 'c#',
 'c++',
 'html',
 'html',
 'android',
 'php',
 'c++',
 'android',
 'php',
 'c#',
 'java',
 'c#',
 'html',
 'python',
 'python',
 'android',
 'html',
 'git',
 'html',
 'java',
 'ios',
 'html',
 'html',
 'git',
 'java',
 'android',
 'c#',
 'javascript',
 'html',
 'html',
 'html',
 'html',
 'javascript',
 'git',
 'ios',
 'ios',
 'php',
 'python',
 'ios',
 'c++',
 'html',
 'android',
 'c++',
 'php',
 'c#',
 'android',
 'android',
 'java',
 'html',
 'android',
 'ios',
 'javascript',
 'javascript',
 'git',
 'htm

In [7]:
from transformers import pipeline 

classifier = pipeline(task = "zero-shot-classification", model = model, use_fast=False)

#sequence_to_classify = "Angela Merkel is a politician in Germany and leader of the CDU"
candidate_labels = TestDF["Label"].unique().tolist()
output = [classifier(sequence_to_classify, candidate_labels, multi_label=False) for sequence_to_classify in TestDF["TitleClean"].values]

In [11]:
TestDF.head()

,index,Date,Title,Tags,Score,Label,TitleClean,TitleCleanTokenised,BertEmbeddingsMean,BertEmbeddingsSum,LabelEncoded
0,8684,2016-07-03 11:03:32,Cross-platform file name handling in .NET Core,"c#, linux, .net-core",59,c#,crossplatform file name handl in net core,"['crossplatform', 'file', 'name', 'handl', 'in...","[-0.15295641124248505, -0.2220861166715622, -0...","[-5.3534746170043945, -7.773014068603516, -0.8...",1
1,7216,2011-06-17 20:44:49,Why use array size 1 instead of pointer?,"c++, c",52,c++,whi use array size 1 instead of pointer,"['whi', 'use', 'array', 'size', '1', 'instead'...","[-0.18913988769054413, -0.2656485438346863, 0....","[-6.430756092071533, -9.032050132751465, 2.569...",2
2,1672,2021-12-21 13:13:27,&quot;Resource not accessible by integration&q...,"git, github-actions, packer",80,git,quotresourc not access by integrationquot on g...,"['quotresourc', 'not', 'access', 'by', 'integr...","[-0.24320729076862335, 0.16181465983390808, 0....","[-8.025840759277344, 5.339883804321289, 13.278...",3
3,3692,2012-03-19 15:16:34,How to use text-overflow ellipsis in an html i...,"html, css, ellipsis",84,html,how to use textoverflow ellipsi in an html inp...,"['how', 'to', 'use', 'textoverflow', 'ellipsi'...","[-0.36745747923851013, -0.19664649665355682, -...","[-10.656267166137695, -5.7027482986450195, -1....",4
4,4095,2012-04-04 10:00:30,java.lang.IllegalStateException: The specified...,"android, android-layout, android-fragments",100,android,javalangillegalstateexcept the specifi child a...,"['javalangillegalstateexcept', 'the', 'specifi...","[-0.29230260848999023, 0.12499813735485077, 0....","[-12.569011688232422, 5.374919891357422, 17.09...",0


In [ ]:
predictions = []

for index, row in TestDF.iterrows():
    predicted_class = classifier(row["Title"], candidate_labels, multi_label=False)

    print(f"Correct Class: {row['Label']} | Predicted Class: {predicted_class['labels'][0]} | Probability: {predicted_class['scores'][0]}")
    predictions.append(predicted_class['labels'][0])


In [18]:
le = LabelEncoder()
le.fit(TestDF["Label"].values)

y = le.transform(TestDF["Label"].values)
y_hate = le.transform(y_hat)

In [30]:
type(y[0])

numpy.int64

In [29]:
type(y_hate[0])

numpy.int64

In [26]:
accuracy_score(y, y_hat)

0.0

In [31]:
from sklearn.metrics import classification_report, accuracy_score
# Evaluate the performance
print("Accuracy on validation set:", accuracy_score(y, y_hate))
print("\nClassification Report:\n", classification_report(y, y_hate))

Accuracy on validation set: 0.6423333333333333

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.52      0.63       300
           1       0.45      0.44      0.45       300
           2       0.56      0.62      0.59       300
           3       0.97      0.89      0.93       300
           4       0.38      0.81      0.52       300
           5       0.91      0.68      0.78       300
           6       0.49      0.66      0.56       300
           7       0.78      0.51      0.62       300
           8       0.84      0.68      0.75       300
           9       0.93      0.61      0.74       300

    accuracy                           0.64      3000
   macro avg       0.71      0.64      0.66      3000
weighted avg       0.71      0.64      0.66      3000



In [16]:
from optimum.onnxruntime import ORTModelForSequenceClassification

ModuleNotFoundError: No module named 'optimum'

In [15]:
from transformers import pipeline
import pandas as pd
import numpy as np

# Example DataFrame creation (replace this with your actual TestDF)
# TestDF = pd.DataFrame({'Title': ["Title 1", "Title 2"], 'Label': ["Label 1", "Label 2"]})

batch_size = 128

# Initialize the classifier pipeline
classifier = pipeline(
    task="zero-shot-classification", 
    model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli",
    tokenizer= "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli",
    use_fast = False,
    batch_size = batch_size
)

# Your unique candidate labels
candidate_labels = TestDF["Label"].unique().tolist()

# Get your sequences
sequences = TestDF["Title"].values.tolist()

# Prepare batches
batches = [sequences[i:i + batch_size] for i in range(0, len(sequences), batch_size)]
BatchNumbers = len(batches)

# Classify each batch and collect the results
results = []
#y_hat =[]

for idx, batch in enumerate(batches):
    # Classify the entire batch at once
    batch_output = classifier(batch, candidate_labels, multi_label = False, batch_size=batch_size)
    results.append(batch_output)  # Collect results
    #y_hat.append(batch_output["labels"][0])
    print(f"Batch {idx+1} of {BatchNumbers}")

Batch 1 of 24
Batch 2 of 24


KeyboardInterrupt: 

In [14]:
batch_output

[{'sequence': 'Cross-platform file name handling in .NET Core',
  'labels': ['c#',
   'c++',
   'git',
   'php',
   'java',
   'html',
   'ios',
   'android',
   'javascript',
   'python'],
  'scores': [0.28499728441238403,
   0.1304446905851364,
   0.10567451268434525,
   0.0951751172542572,
   0.09144368022680283,
   0.079079270362854,
   0.07131393253803253,
   0.06028226017951965,
   0.04144822806119919,
   0.04014101251959801]},
 {'sequence': 'Why use array size 1 instead of pointer?',
  'labels': ['c++',
   'php',
   'html',
   'java',
   'c#',
   'python',
   'ios',
   'git',
   'javascript',
   'android'],
  'scores': [0.18912053108215332,
   0.14287739992141724,
   0.1188075840473175,
   0.11667409539222717,
   0.10041195154190063,
   0.09070038795471191,
   0.06534364074468613,
   0.06445636600255966,
   0.060079872608184814,
   0.05152808502316475]},
 {'sequence': '&quot;Resource not accessible by integration&quot; on github post /repos/{owner}/{repo}/actions/runners/registr

In [12]:
batch_output

[{'sequence': 'Cross-platform file name handling in .NET Core',
  'labels': ['c#',
   'c++',
   'git',
   'php',
   'java',
   'html',
   'ios',
   'android',
   'javascript',
   'python'],
  'scores': [0.28499728441238403,
   0.1304446905851364,
   0.10567451268434525,
   0.0951751172542572,
   0.09144368022680283,
   0.079079270362854,
   0.07131393253803253,
   0.06028226017951965,
   0.04144822806119919,
   0.04014101251959801]},
 {'sequence': 'Why use array size 1 instead of pointer?',
  'labels': ['c++',
   'php',
   'html',
   'java',
   'c#',
   'python',
   'ios',
   'git',
   'javascript',
   'android'],
  'scores': [0.18912053108215332,
   0.14287739992141724,
   0.1188075840473175,
   0.11667409539222717,
   0.10041195154190063,
   0.09070038795471191,
   0.06534364074468613,
   0.06445636600255966,
   0.060079872608184814,
   0.05152808502316475]},
 {'sequence': '&quot;Resource not accessible by integration&quot; on github post /repos/{owner}/{repo}/actions/runners/registr

In [122]:
help(classifier)

Help on ZeroShotClassificationPipeline in module transformers.pipelines.zero_shot_classification object:

class ZeroShotClassificationPipeline(transformers.pipelines.base.ChunkPipeline)
 |  ZeroShotClassificationPipeline(args_parser=<transformers.pipelines.zero_shot_classification.ZeroShotClassificationArgumentHandler object at 0x17587ef90>, *args, **kwargs)
 |  
 |  NLI-based zero-shot classification pipeline using a `ModelForSequenceClassification` trained on NLI (natural
 |  language inference) tasks. Equivalent of `text-classification` pipelines, but these models don't require a
 |  hardcoded number of potential classes, they can be chosen at runtime. It usually means it's slower but it is
 |  **much** more flexible.
 |  
 |  Any combination of sequences and labels can be passed and each combination will be posed as a premise/hypothesis
 |  pair and passed to the pretrained model. Then, the logit for *entailment* is taken as the logit for the candidate
 |  label being valid. Any NL

In [121]:
classifier(batch, [candidate_labels[0] for _ in range(len(batch))])

{'sequence': 'Cross-platform file name handling in .NET Core',
 'labels': ['c#', 'c#', 'c#', 'c#'],
 'scores': [0.25, 0.25, 0.25, 0.25]}

In [81]:
batch_output

{'sequence': 'Cross-platform file name handling in .NET Core',
 'labels': ['c#',
  'c++',
  'git',
  'php',
  'java',
  'html',
  'ios',
  'android',
  'javascript',
  'python'],
 'scores': [0.2849971055984497,
  0.13044464588165283,
  0.10567457228899002,
  0.09517516940832138,
  0.09144382923841476,
  0.07907922565937042,
  0.07131394743919373,
  0.060282256454229355,
  0.04144821688532829,
  0.040141038596630096]}

In [92]:
from transformers import pipeline
import numpy as np
batch_size = 4

# Initialize the classifier pipeline
classifier = pipeline(
    task="zero-shot-classification", 
    model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli",
    tokenizer= "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli",
    use_fast=False,
)

# Your unique candidate labels
candidate_labels = TestDF["Label"].unique().tolist()

# Define your batch size
  # Adjust the batch size according to your memory capacity

# Get your sequences
sequences = TestDF["Title"].values

# Prepare batches
batches = [sequences[i:i + batch_size] for i in range(0, len(sequences), batch_size)]
BatchNumbers = len(batches)

# Classify each batch and collect the results
results = []

for idx, batch in enumerate(batches):
    for sequence in batch:
      all_model_outputs = []
      for preprocessed in classifier.preprocess(sequence, candidate_labels):
        model_outputs = classifier.forward(preprocessed)
        all_model_outputs.append(model_outputs)
      results.append(classifier.postprocess(all_model_outputs))
    #batch_output = classifier(batch, candidate_labels, multi_label=True)
    #results.append(batch_output)
    print(f"Batch {idx} of {BatchNumbers}")
    break

outputs = classifier.postprocess(all_model_outputs, batch_size)

Batch 0 of 750


In [110]:
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", use_fast = False)

In [ ]:
premise

tokenizer(premise, hypothesis, return_tensors="pt")

In [109]:
help(classifier.preprocess)

Help on method preprocess in module transformers.pipelines.zero_shot_classification:

preprocess(inputs, candidate_labels=None, hypothesis_template='This example is {}.') method of transformers.pipelines.zero_shot_classification.ZeroShotClassificationPipeline instance
    Preprocess will take the `input_` of a specific pipeline and return a dictionary of everything necessary for
    `_forward` to run properly. It should contain at least one tensor, but might have arbitrary other items.



In [106]:

import torch
inputs = [preprocessed for preprocessed in classifier.preprocess(sequence, candidate_labels)]
input_ids = torch.stack([item['input_ids'] for item in inputs])
token_type_ids = torch.stack([item['token_type_ids'] for item in inputs])
attention_mask = torch.stack([item['attention_mask'] for item in inputs])

RuntimeError: stack expects each tensor to be equal size, but got [1, 22] at entry 0 and [1, 23] at entry 1

In [103]:
for i in classifier.preprocess(batch, [candidate_labels]):
    print(i)

{'candidate_label': ['c#', 'c++', 'git', 'html', 'android', 'ios', 'python', 'javascript', 'java', 'php'], 'sequence': 'Cross-platform file name handling in .NET Core', 'is_last': True, 'input_ids': tensor([[    1,  4241,   271, 22028,  1092,   601,  3957,   267,   323,  9208,
          6708,     2,   329,   738,   269,   647,   280,  1207,  2669,   280,
           261,   382,  1207,  1186,  1186,   280,   261,   382, 29659,   280,
           261,   382,  6338,   280,   261,   382, 62859,   280,   261,   382,
         20159,   280,   261,   382, 76502,   280,   261,   382, 40631, 20749,
           280,   261,   382, 40631,   280,   261,   382, 10430,   280,   592,
           260,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [95]:
tmp = [preprocessed for sequence in batch[0] for preprocessed in classifier.preprocess(sequence, candidate_labels)]
tmp

[{'candidate_label': 'c#',
  'sequence': 'C',
  'is_last': False,
  'input_ids': tensor([[   1,  716,    2,  329,  738,  269, 2285, 2669,  260,    2]]),
  'token_type_ids': tensor([[0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'candidate_label': 'c++',
  'sequence': 'C',
  'is_last': False,
  'input_ids': tensor([[   1,  716,    2,  329,  738,  269, 2285, 1186, 1186,  260,    2]]),
  'token_type_ids': tensor([[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'candidate_label': 'git',
  'sequence': 'C',
  'is_last': False,
  'input_ids': tensor([[    1,   716,     2,   329,   738,   269, 25182,   260,     2]]),
  'token_type_ids': tensor([[0, 0, 0, 1, 1, 1, 1, 1, 1]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'candidate_label': 'html',
  'sequence': 'C',
  'is_last': False,
  'input_ids': tensor([[    1,   716,     2,   329,   738,   269, 22994,   260,     2]]

In [99]:
torch.stack([i["input_ids"] for i in tmp])

RuntimeError: stack expects each tensor to be equal size, but got [1, 10] at entry 0 and [1, 11] at entry 1

In [90]:
tmp = [preprocessed for sequence in batch for preprocessed in classifier.preprocess(sequence, candidate_labels)]
classifier.forward(tmp)

TypeError: list indices must be integers or slices, not str

In [83]:
for preprocessed in classifier.preprocess(batch, candidate_labels):
    print(preprocessed)

{'candidate_label': 'c#', 'sequence': 'Cross-platform file name handling in .NET Core', 'is_last': False, 'input_ids': tensor([[    1,  4241,   271, 22028,  1092,   601,  3957,   267,   323,  9208,
          6708,     2,   329,   738,   269,  2285,  2669,   260,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'candidate_label': 'c++', 'sequence': 'Cross-platform file name handling in .NET Core', 'is_last': False, 'input_ids': tensor([[    1,  4241,   271, 22028,  1092,   601,  3957,   267,   323,  9208,
          6708,     2,   329,   738,   269,  2285,  1186,  1186,   260,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'candidate_label': 'git', 'sequence': 'Cross-platform file name handling in .NET Core', 'is

In [67]:
help(classifier.postprocess)

Help on method postprocess in module transformers.pipelines.zero_shot_classification:

postprocess(model_outputs, multi_label=False) method of transformers.pipelines.zero_shot_classification.ZeroShotClassificationPipeline instance
    Postprocess will receive the raw outputs of the `_forward` method, generally tensors, and reformat them into
    something more friendly. Generally it will output a list or a dict or results (containing just strings and
    numbers).



In [66]:
TestDF.head()

,index,Date,Title,Tags,Score,Label,TitleClean,TitleCleanTokenised,BertEmbeddingsMean,BertEmbeddingsSum,LabelEncoded
0,8684,2016-07-03 11:03:32,Cross-platform file name handling in .NET Core,"c#, linux, .net-core",59,c#,crossplatform file name handl in net core,"['crossplatform', 'file', 'name', 'handl', 'in...","[-0.15295641124248505, -0.2220861166715622, -0...","[-5.3534746170043945, -7.773014068603516, -0.8...",1
1,7216,2011-06-17 20:44:49,Why use array size 1 instead of pointer?,"c++, c",52,c++,whi use array size 1 instead of pointer,"['whi', 'use', 'array', 'size', '1', 'instead'...","[-0.18913988769054413, -0.2656485438346863, 0....","[-6.430756092071533, -9.032050132751465, 2.569...",2
2,1672,2021-12-21 13:13:27,&quot;Resource not accessible by integration&q...,"git, github-actions, packer",80,git,quotresourc not access by integrationquot on g...,"['quotresourc', 'not', 'access', 'by', 'integr...","[-0.24320729076862335, 0.16181465983390808, 0....","[-8.025840759277344, 5.339883804321289, 13.278...",3
3,3692,2012-03-19 15:16:34,How to use text-overflow ellipsis in an html i...,"html, css, ellipsis",84,html,how to use textoverflow ellipsi in an html inp...,"['how', 'to', 'use', 'textoverflow', 'ellipsi'...","[-0.36745747923851013, -0.19664649665355682, -...","[-10.656267166137695, -5.7027482986450195, -1....",4
4,4095,2012-04-04 10:00:30,java.lang.IllegalStateException: The specified...,"android, android-layout, android-fragments",100,android,javalangillegalstateexcept the specifi child a...,"['javalangillegalstateexcept', 'the', 'specifi...","[-0.29230260848999023, 0.12499813735485077, 0....","[-12.569011688232422, 5.374919891357422, 17.09...",0


In [65]:
all_model_outputs

[{'candidate_label': 'c#',
  'sequence': 'Cross-platform file name handling in .NET Core',
  'is_last': False,
  'logits': tensor([[-1.5839,  0.8148,  0.8260]])},
 {'candidate_label': 'c++',
  'sequence': 'Cross-platform file name handling in .NET Core',
  'is_last': False,
  'logits': tensor([[-2.3655,  1.8767,  0.4887]])},
 {'candidate_label': 'git',
  'sequence': 'Cross-platform file name handling in .NET Core',
  'is_last': False,
  'logits': tensor([[-2.5760,  3.8888, -1.4911]])},
 {'candidate_label': 'html',
  'sequence': 'Cross-platform file name handling in .NET Core',
  'is_last': False,
  'logits': tensor([[-2.8660,  3.0390, -0.1966]])},
 {'candidate_label': 'android',
  'sequence': 'Cross-platform file name handling in .NET Core',
  'is_last': False,
  'logits': tensor([[-3.1374,  4.4188, -1.2766]])},
 {'candidate_label': 'ios',
  'sequence': 'Cross-platform file name handling in .NET Core',
  'is_last': False,
  'logits': tensor([[-2.9693,  4.2819, -1.3395]])},
 {'candidate

In [60]:
for preprocessed in classifier.preprocess(batch, batch_size):
    print(preprocessed)

TypeError: object of type 'int' has no len()

In [35]:
pipeline

<function transformers.pipelines.pipeline(task: str = None, model: Union[str, ForwardRef('PreTrainedModel'), ForwardRef('TFPreTrainedModel'), NoneType] = None, config: Union[str, transformers.configuration_utils.PretrainedConfig, NoneType] = None, tokenizer: Union[str, transformers.tokenization_utils.PreTrainedTokenizer, ForwardRef('PreTrainedTokenizerFast'), NoneType] = None, feature_extractor: Union[str, ForwardRef('SequenceFeatureExtractor'), NoneType] = None, image_processor: Union[str, transformers.image_processing_utils.BaseImageProcessor, NoneType] = None, framework: Optional[str] = None, revision: Optional[str] = None, use_fast: bool = True, token: Union[str, bool, NoneType] = None, device: Union[int, str, ForwardRef('torch.device'), NoneType] = None, device_map=None, torch_dtype=None, trust_remote_code: Optional[bool] = None, model_kwargs: Dict[str, Any] = None, pipeline_class: Optional[Any] = None, **kwargs) -> transformers.pipelines.base.Pipeline>

In [48]:
# Initialize the classifier pipeline
classifier = pipeline(
    task="zero-shot-classification", 
    model_card="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", 
    use_fast=False, batch_size = 128
)

classifier(["How do I iterate over a list?"], candidate_labels, multi_label = True)


[{'sequence': 'How do I iterate over a list?',
  'labels': ['python',
   'php',
   'java',
   'html',
   'javascript',
   'ios',
   'c#',
   'c++',
   'git',
   'android'],
  'scores': [0.22552040219306946,
   0.19005393981933594,
   0.1718694120645523,
   0.15656940639019012,
   0.11534187942743301,
   0.11111412197351456,
   0.10964552313089371,
   0.09744724631309509,
   0.08610676229000092,
   0.04532221704721451]}]

In [44]:

id2label = {0:'python', 1:'php', 2:'java', 3:'html', 4:'javascript', 5:'ios', 6:'c#', 7:'c++', 8:'git', 9:'android'}
label2id = {'python':0, 'php':1, 'java':2, 'html':3, 'javascript':4, 'ios':5, 'c#':6, 'c++':7, 'git':8, 'android':9}

In [91]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model_name ="microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name , num_labels = 10, id2label=id2label, label2id=label2id)

premise = "Classify the following input text into one of the following ten categories: ['python', 'php', 'java', 'html', 'javascript', 'ios', 'c#', 'c++', 'git', 'android']"
hypothesis = "'&quot;Resource not accessible by integration&quot; on github post /repos/{owner}/{repo}/actions/runners/registration-token API'"

input = tokenizer(premise, hypothesis, return_tensors="pt")
output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names =['python', 'php', 'java', 'html', 'javascript', 'ios', 'c#', 'c++', 'git', 'android']
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)

/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'python': 9.2, 'php': 11.9, 'java': 9.6, 'html': 11.2, 'javascript': 9.6, 'ios': 8.7, 'c#': 8.7, 'c++': 9.6, 'git': 10.6, 'android': 10.9}


In [29]:
len(results)

24

In [14]:
classifier(TestDF["Title"].values[2], candidate_labels, multi_label=False)

{'sequence': '&quot;Resource not accessible by integration&quot; on github post /repos/{owner}/{repo}/actions/runners/registration-token API',
 'labels': ['git',
  'html',
  'python',
  'c#',
  'c++',
  'java',
  'javascript',
  'php',
  'android',
  'ios'],
 'scores': [0.46193355321884155,
  0.13478106260299683,
  0.10520695894956589,
  0.09447404742240906,
  0.07192490249872208,
  0.04554850980639458,
  0.030028270557522774,
  0.029239516705274582,
  0.016867972910404205,
  0.00999519694596529]}

In [9]:
TestDF["Title"].values[0]

'Cross-platform file name handling in .NET Core'

In [5]:
TestDF["TitleClean"].values

NameError: name 'output' is not defined

In [6]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")


In [56]:
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier

In [69]:
param_grid = {
    'n_neighbors': list(range(1, 31)),  # Number of neighbors to consider
    'weights': ['uniform', 'distance'],  # Uniform weights or weighted by distance
    'metric': ['euclidean', 'manhattan', 'minkowski', 'cosine'],  # Different distance metrics
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute the nearest neighbors
    'leaf_size': list(range(10, 51, 10)),  # Leaf size passed to BallTree or KDTree
    #'p': [1, 2]  # Power parameter for the Minkowski metric
}


# Set up the classifier
knn = KNeighborsClassifier()

# Set up the cross-validation strategy
# For example, 5-fold cross-validation
cv_strategy = 5

# Set up the grid search with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=cv_strategy, verbose=1, n_jobs=-1)

# Fit the grid search to the data
# old_question_embeddings and old_question_classes should be defined based on your labeled data
grid_search.fit(np.array(TrainDF["BertEmbeddingsMean"].values.tolist()), np.array(TrainDF["LabelEncoded"].values.tolist()))

# The best hyperparameters from the grid search
best_params = grid_search.best_params_

# The best model from the grid search
best_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best parameters found: ", best_params)


Fitting 5 folds for each of 4800 candidates, totalling 24000 fits


/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
3000 fits failed out of a total of 24000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1500 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/sklearn/base.py", line 1152, in wrapper

Best parameters found:  {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'cosine', 'n_neighbors': 29, 'weights': 'distance'}


In [70]:
from sklearn.metrics import classification_report, accuracy_score

# Assuming validation_data is your preprocessed validation dataset
# and validation_labels are the true labels for the validation dataset
validation_labels = np.array(TestDF["LabelEncoded"].values.tolist())

# Generate embeddings for the validation data
validation_embeddings = np.array(TestDF["BertEmbeddingsMean"].values.tolist())

# Use the best model to make predictions on the validation data
predictions = best_model.predict(validation_embeddings)

# Evaluate the performance
print("Accuracy on validation set:", accuracy_score(validation_labels, predictions))
print("\nClassification Report:\n", classification_report(validation_labels, predictions))

Accuracy on validation set: 0.506

Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.62      0.55       300
           1       0.35      0.35      0.35       300
           2       0.44      0.49      0.46       300
           3       0.54      0.78      0.64       300
           4       0.64      0.66      0.65       300
           5       0.49      0.53      0.51       300
           6       0.43      0.34      0.38       300
           7       0.47      0.28      0.35       300
           8       0.69      0.62      0.65       300
           9       0.52      0.38      0.44       300

    accuracy                           0.51      3000
   macro avg       0.51      0.51      0.50      3000
weighted avg       0.51      0.51      0.50      3000



In [ ]:
import joblib

# Save the model to a file
joblib.dump(best_model, 'best_knn_model.joblib')

# Load the model from the file
best_model_loaded = joblib.load('best_knn_model.joblib')


In [67]:
# Build a Nearest Neighbors model
nbrs = KNeighborsClassifier(n_neighbors=40,weights="distance", metric='cosine', n_jobs=-1).fit(np.array(TrainDF["BertEmbeddingsMean"].values.tolist()), np.array(TrainDF["LabelEncoded"].values.tolist()))

#grid_search.fit(np.array(TrainDF["BertEmbeddings"].values.tolist()), np.array(TrainDF["LabelEncoded"].values.tolist()))


In [68]:
predicted_class = nbrs.predict(np.array(TestDF["BertEmbeddingsMean"].values.tolist()))

accuracy_score(np.array(TestDF["LabelEncoded"].values.tolist()), predicted_class)

0.5016666666666667

In [42]:
# Find the most similar old question
distances, indices = nbrs.kneighbors(np.array(TestDF["BertEmbeddings"].values.tolist()))

In [22]:
indices

array([[1203,  959,  936, ...,  874,  767,  886],
       [1221, 1377,  761, ..., 1218, 1164,  712],
       [ 761,  932, 1377, ..., 1017,  510, 1221],
       ...,
       [ 239,  545,  675, ...,  743,    0,  689],
       [1224,  716,  994, ..., 1114, 1384,  978],
       [ 231, 1053,  891, ..., 1085,  475, 1255]])

In [32]:
TestDF.loc[0, "LabelEncoded"]

0

In [28]:
TrainDF.loc[indices[0], "LabelEncoded"].value_counts().to_dict()

{0: 7, 1: 3}

In [43]:
predicted_class = []

# Perform a majority vote for classification
for i in range(TestDF.shape[0]):
    neighbor_indices = indices[i]
    neighbor_classes = TrainDF.loc[indices[i], "Label"].value_counts().to_dict() 
    neighbor_classes_encoded = TrainDF.loc[indices[i], "LabelEncoded"].value_counts().to_dict()
    predicted_class.append(list(neighbor_classes_encoded.keys())[0])
    #targetTestDF.loc[0, "LabelEncoded"]



    #print(f"The new question '{new_q}' is classified as: '{most_common_class}' based on majority vote")

In [44]:
accuracy_score(np.array(TestDF["LabelEncoded"].values.tolist()), predicted_class)

0.7516666666666667